In [1]:
import os
import datetime
import pandas as pd
import csv

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY= os.environ.get('API_CODE')
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [ ]:
# read file columns with panda
df = pd.read_csv('../minecraft_ch_data_clean.json')